In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Load the pre-trained ResNet model
resnet = models.resnet50(pretrained=True)
# Remove the classification layer (final fully connected layer)
resnet = nn.Sequential(*list(resnet.children())[:-1])
# Set the model to evaluation mode
resnet.eval()

In [ ]:
# Function to preprocess the image and extract features using ResNet
def extract_features(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),      # Resize the image to (224, 224) for ResNet input
        transforms.ToTensor(),              # Convert image to tensor
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize the image
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        features = resnet(image)
    return features.squeeze()

# Example usage
image_path = "/content/google.jpg"
image_features = extract_features(image_path)
     

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define the RNN-based captioning model
class CaptioningModel(nn.Module):
    def __init__(self, input_size, hidden_size, vocab_size):
        super(CaptioningModel, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, input_size)
        self.rnn = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, image_features, captions):
        embeds = self.embedding(captions)
        embeds = torch.cat((image_features.unsqueeze(0), embeds), dim=0)
        lstm_out, _ = self.rnn(embeds)
        output = self.fc(lstm_out)
        return output

# Example usage
input_size = 512  # ResNet output size
hidden_size = 256  # Size of the hidden layer in LSTM
vocab_size = 10000  # The size of the vocabulary (number of unique words)
captioning_model = CaptioningModel(input_size, hidden_size, vocab_size)
     